In [2]:
import torch
import numpy as np
from tqdm import tqdm

In [3]:

# Define alphas as torch tensors with requires_grad=True
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

alphas = [alpha_1, alpha_2, alpha_3, alpha_4]

# psi function using torch
def psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Tensors with gradients enabled
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y, *alphas)

# First gradients w.r.t. x and y
grad_x = torch.autograd.grad(psi_val, x, create_graph=True)[0]
grad_y = torch.autograd.grad(psi_val, y, create_graph=True)[0]

# Laplacians
laplacian_x = sum(torch.autograd.grad(grad_x[i], x, retain_graph=True, create_graph=True)[0][i] for i in range(3))
laplacian_y = sum(torch.autograd.grad(grad_y[i], y, retain_graph=True, create_graph=True)[0][i] for i in range(3))

lap = laplacian_x + laplacian_y
# Derivatives of Laplacians w.r.t. alphas
dlap_dalpha = torch.autograd.grad(lap, alphas, retain_graph=True)


print(dlap_dalpha)

(tensor(-0.5851, dtype=torch.float64), tensor(2.5131, dtype=torch.float64), tensor(0.6771, dtype=torch.float64), tensor(-2.5998, dtype=torch.float64))


In [4]:
"""
def local_energy(psi_val, r1, r2):
    grad_x = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_y = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    # Laplacians
    laplacian_x = sum(torch.autograd.grad(grad_x[i], r1, retain_graph=True, create_graph=True)[0][i] for i in range(3))
    laplacian_y = sum(torch.autograd.grad(grad_y[i], r2, retain_graph=True, create_graph=True)[0][i] for i in range(3))

    ke = - 0.5 * (laplacian_x + laplacian_y)

    return - 2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2) + ke
"""

def local_energy(psi_val, r1, r2):
    grad_r1 = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_r2 = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    lap_r1 = torch.autograd.grad(grad_r1, r1, grad_outputs=torch.ones_like(grad_r1), create_graph=True)[0]
    lap_r2 = torch.autograd.grad(grad_r2, r2, grad_outputs=torch.ones_like(grad_r2), create_graph=True)[0]

    laplacian = lap_r1.sum() + lap_r2.sum()
    ke = -0.5 * laplacian

    potential = -2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2)

    return ke + potential


In [7]:
L = 1
r1 = torch.rand(3, requires_grad=True) * 2 * L - L
r2 = torch.rand(3, requires_grad=True) * 2 * L - L #random number from -L to L
E = 0
E2 = 0
Eln_average = 0
ln_average = 0
rejection_ratio = 0
step = 0
max_steps = 500
N = 10000
dlap_dalpha = 0
xs = []
ys = []
psi_vals = []

for i in tqdm(range(N)):
    flag = False
    chose = np.random.rand()
    step = step + 1
    if chose < 0.5:
        r1_trial = r1 + 0.5 * (torch.rand(3) * 2 * L-L)
        r2_trial = r2
    else:
        r2_trial = r2 + 0.5 * (torch.rand(3) * 2 * L-L)
        r1_trial = r1


    psi_val = psi(r1, r2, *alphas)
    psi_val_trial = psi(r1_trial, r2_trial, *alphas)
    

    if psi_val_trial.item() >= psi_val.item():
        r1 = r1_trial
        r2 = r2_trial
        flag = True
    
    else:
        dummy = np.random.rand()
        if dummy < psi_val_trial.item() / psi_val.item():
            r1 = r1_trial
            r2 = r2_trial
        else:
            rejection_ratio += 1./N
            
    if step > max_steps:
        xs.append(r1)
        ys.append(r2)

        if flag:
            psi_vals.append(psi_val_trial)
        else:
            psi_vals.append(psi_val)
        """
        local_E = local_energy(psi_val_trial, r1, r2)
        E += local_E / (N - max_steps)
        E2 += local_E ** 2 / (N - max_steps)
        dlap_dalpha += torch.tensor(torch.autograd.grad(local_E, alphas, retain_graph=True)) / (N - max_steps)
        """





100%|██████████| 10000/10000 [00:04<00:00, 2314.54it/s]


In [8]:
def batched_psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x, dim=1)
    r2 = torch.norm(y, dim=1)
    r12 = torch.norm(x - y, dim=1)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [35]:
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

def batched_psi(x, y):
    r1 = torch.norm(x, dim=1)
    r2 = torch.norm(y, dim=1)
    r12 = torch.norm(x - y, dim=1)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3


In [ ]:
vmap(psi)

In [38]:
t = batched_psi(torch.stack(xs), torch.stack(ys))

In [40]:
torch.autograd.grad(t, torch.stack(xs[:5]), is_grads_batched=True)

RuntimeError: grad can be implicitly created only for scalar outputs

In [16]:
from torch.func import vmap, grad

In [33]:
batched_grad_f = vmap(grad(batched_psi))

In [59]:
jacobian = torch.autograd.functional.jacobian(batched_psi, (torch.stack(xs), torch.stack(ys)))

In [60]:
jacobian[0].shape

torch.Size([9500, 9500, 3])

In [63]:
from torch.func import grad, vmap

In [162]:
import torch
from torch.autograd.functional import jacobian

# Original scalar-valued psi
def psi(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [181]:

def third_partial_derivatives(scalar_func, inp):
    return jacobian(
        lambda x: jacobian(
            lambda y: jacobian(scalar_func, y), 
            x
        ), 
        inp
    )

In [171]:
t = vmap(psi)

In [177]:
inputs = []
for i in range(len(xs)):
    inputs.append(torch.tensor([*xs[i], *ys[i], alpha_1, alpha_2, alpha_3, alpha_4]))

In [178]:
inputs = torch.stack(inputs)

In [180]:
inputs[0]

tensor([-0.7768,  0.0990,  0.2629, -1.0013,  2.1483,  0.9251,  1.0130,  0.2119,
         0.1406,  0.0030], dtype=torch.float64)

In [176]:
t(inputs)

tensor([0.0039, 0.0046, 0.0035,  ..., 0.0008, 0.0013, 0.0020],
       dtype=torch.float64)

In [100]:
s = vmap(grad(psi))

In [120]:
def fs(x):
    return torch.exp(x[0] + x[1])

In [121]:
l = vmap(grad(grad(fs)))

In [127]:
l = vmap(grad(grad(fs)))

In [114]:
grad(grad(fs))(torch.tensor(3.))

tensor(20.0855)

In [ ]:
t = vmap(hessian(psi))
E = torch.diagonal(hess_full[0]).sum()
grad(E, alphas)

In [145]:
hess_full = t(inputs)

In [ ]:
hess_full[0].shape 

torch.Size([6, 6])

In [155]:
E = torch.diagonal(hess_full[0]).sum()

In [160]:
lap_alpha = jacobian(hessian(psi))

TypeError: jacobian() missing 1 required positional argument: 'inputs'

In [159]:
lap_alpha(inputs[0])

RuntimeError: grad_and_value(f)(*args): Expected f(*args) to return a scalar Tensor, got tensor with 2 dims. Maybe you wanted to use the vjp or jacrev APIs instead?